# Size a HPP using EGO parallel 


This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on parallel evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

### Design Variables

**Wind Plant design:**

- Number of wind turbines in the wind plant [-] (`Nwt`)
- Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

- Solar plant power capacity [MW] (`solar_MW`)

**Battery Storage design:**

- Battery power [MW] (`b_P`)
- Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
- Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

### Example sites



In [2]:
import pandas as pd
from hydesign.examples import examples_filepath
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0)
examples_sites

,case,name,longitude,latitude,altitude,input_ts_fn,sim_pars_fn,price_fn,price_col
0,India,Indian_site_good_wind,77.500226,8.334294,679.803454,India/input_ts_Indian_site_good_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
1,India,Indian_site_good_solar,68.542204,23.542099,29.883557,India/input_ts_Indian_site_good_solar.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
2,India,Indian_site_bad_solar_bad_wind,77.916878,17.292316,627.424643,India/input_ts_Indian_site_bad_solar_bad_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
3,Europe,France_good_solar,4.229736,44.422011,204.000000,Europe/input_ts_France_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
4,Europe,France_good_wind,-0.864258,48.744116,302.000000,Europe/input_ts_France_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
5,Europe,France_bad_solar_n_wind,2.167969,47.428087,140.000000,Europe/input_ts_France_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
6,Europe,Germany_bad_solar_n_wind,10.766602,49.310798,442.000000,Europe/input_ts_Germany_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_ME
7,Europe,Germany_good_wind,7.873535,53.287111,5.000000,Europe/input_ts_Germany_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_NW
8,Europe,Denmark_good_solar,11.813965,55.397760,42.000000,Europe/input_ts_Denmark_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_E
9,Europe,Denmark_good_wind,8.594398,56.227322,85.000000,Europe/input_ts_Denmark_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_W


### Running Parallel EGO



In [3]:
from hydesign.Parallel_EGO import EfficientGlobalOptimizationDriver, get_kwargs
from smt.applications.mixed_integer import (
    MixedIntegerContext,
    FLOAT,
    INT,)
import numpy as np
from hydesign.hpp_assembly import hpp_model

In [4]:
inputs = {
    'example': 9,
    'name': None,
    'longitude': None,
    'latitude': None,
    'altitude': None,
    'input_ts_fn': None,
    'sim_pars_fn': None,

    'opt_var': "NPV_over_CAPEX",
    'rotor_diameter_m': 100,
    'hub_height_m': 120,
    'wt_rated_power_MW': 2,
    'surface_tilt_deg': 20,
    'surface_azimuth_deg': 180,
    'DC_AC_ratio': 1,
    'num_batteries': 2,
    'n_procs': 31,
    'n_doe': 128,
    'n_clusters': 4,
    'n_seed': 0,
    'max_iter': 15,
    'final_design_fn': 'hydesign_output_9.csv',
    'npred': 2e4, #1e5, #3e4, 2e4
    'tol': 1e-6,
    'min_conv_iter': 3,
    'work_dir': './',
    }

kwargs = get_kwargs(inputs)
     
kwargs['variables'] = {
    'clearance [m]':
        {'var_type':'design',
         'limits':[10, 60],
         'types':INT
         },
     'sp [m2/W]':
        {'var_type':'design',
         'limits':[200, 360],
         'types':INT
         },
    'p_rated [MW]':
        {'var_type':'design',
         'limits':[1, 10],
         'types':INT
         },
    'Nwt':
        {'var_type':'design',
         'limits':[0, 400],
         'types':INT
         },
    'wind_MW_per_km2 [MW/km2]':
        {'var_type':'design',
         'limits':[5, 9],
         'types':FLOAT
         },
    'solar_MW [MW]':
        {'var_type':'design',
         'limits':[0, 400],
         'types':INT
         },
    'surface_tilt [deg]':
        {'var_type':'design',
         'limits':[0, 50],
         'types':FLOAT
         },
    'surface_azimuth [deg]':
        {'var_type':'design',
         'limits':[150, 210],
         'types':FLOAT
         },
    'DC_AC_ratio':
        {'var_type':'design',
         'limits':[1, 2.0],
         'types':FLOAT
         },
    # 'DC_AC_ratio':
    #     {'var_type':'fixed',
    #      'value':1.0,
    #      },
    'b_P [MW]':
        {'var_type':'design',
         'limits':[0, 10],
         'types':INT
         },
    'b_E_h [h]':
        {'var_type':'design',
         'limits':[1, 10],
         'types':INT
         },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        {'var_type':'design',
         'limits':[0, 20],
         'types':FLOAT
         },
} 

EGOD = EfficientGlobalOptimizationDriver(model=hpp_model, **kwargs)
EGOD.run()
result = EGOD.result

Selected example site:
---------------------------------------------------
case                                          Europe
name                               Denmark_good_wind
longitude                                   8.594398
latitude                                   56.227322
altitude                                        85.0
input_ts_fn    Europe/input_ts_Denmark_good_wind.csv
sim_pars_fn                      Europe/hpp_pars.yml
price_fn                    Europe/2030-EL_PRICE.csv
price_col                                       DK_W
Name: 9, dtype: object




Sizing a HPP plant at Denmark_good_wind:


Fixed parameters on the site
-------------------------------
longitude = 8.594398
latitude = 56.227322
altitude = 85.0



Initial 128 simulations took 1.27 minutes
  Current solution -NPV_over_CAPEX = -3.586E-01

Update sm and extract candidate points took 0.08 minutes
Check-optimal candidates: new 28 simulations took 0.26 minutes
  Current solution -NPV_over_CAPEX = -3.833E-